# Technical note: Floats, units, and pacing

## Equality in floats

The floating point representation of (most) real numbers is an approximation of their true value, and when manipulating floats the error in this approximation can build up with each operation.
In most cases this is OK, e.g. when solving a (non-chaotic) ODE, but it does lead to complications in a few areas of Myokit.
Notably, when comparing if the current simulation time equals a predetermined time set in some pacing protocol, and when comparing if the unit resulting from some calculation equals another unit.

Four types of "equality" come into play:

- `x is y`: variables x and y point to the same value in memory.
- `x == y`: variables x and y have the same (binary) floating point representation
- `myokit.float.eq(x, y)`: variables x and y have the same value, to "within floating point precision" (i.e. the difference between them is what we'd expect after a single floating point operation, e.g. x = 1 but y is a sum of two floats that equals 1.).
- `myokit.float.close(x, y, reltol=1e-9, abstol=1e-9)`: variables x and y are equal to within some user-defined (and relatively coarse) tolerance.

## Implementation in Python and Cm

### eq

The ``eq`` method is implemented for use in the pacing system in both Python and C.
A second ``geq`` method implements greater-than-or-equal using ``eq`` for the equality check.

In each language, the ``eq(a, b)`` method:
1. Finds an appropriate ``scale``, by taking the largest absolute value of ``a`` or ``b``: ``scale = max(|a|, |b|)``.
2. Checks for simple equality or, failing that, for ``|a - b| < scale * epsilon``.

A Python implementation is

```
def eq(a, b):
    return a == b or abs(a - b) < max(abs(a), abs(b)) * sys.float_info.epsilon
```

A C implementation is

```
#define scale(a, b) (fabs(a) > fabs(b) ? fabs(a) : fabs(b))
#define eq(a, b) ((a == b) || (fabs(a - b) / scale(a, b) < DBL_EPSILON))
```

### close

This method is only implemented in Python, for use in the units system.

A Python implementation is

```
def close(a, b, reltol=1e-9, abstol=1e-9):
    return a == b or abs(a - b) < max(reltol * max(abs(a), abs(b)), abstol)
```

For further methods based on ``eq`` and ``close``, see the [myokit.float documentation](https://myokit.readthedocs.io/en/stable/api_core/float.html)

## Use in pacing

There are two areas in Myokit's event-based pacing where floating point precision comes into play:

1. During simulations, when using event-based pacing.
   1. For simulations in C, this is handled via `pacing.h`
   2. For simulations in Python, this is handled via a [PacingSystem](https://myokit.readthedocs.io/en/stable/api_simulations/Protocol.html#myokit.PacingSystem)
2. When parsing protocols that use the `next` keyword.

### Floating points in event-based pacing during simulations

1. To determine if an event has started or stopped, times are compared with `geq`.
This means an event will start if `t_sim` is within floating point precision of `t_start`.

2. Once an event has finished, the pacing system will check if another event is due to start by comparing `t_end_1` with `t_start_1` using `eq`, i.e. to within machine precision.


### Floating points in event-based protocol parsing using `next`

In the `mmt` protocol syntax, the `next` keyword can be used to have an event start after the previous event ended, without explicitly specifying this start time.

In Myokit 1.38 and newer, start times set with `next` are rounded to within 9 digits.
This helps avoid floating point inaccuracies building up in long sequences of events using `next`.
The change was introduced in [PR 1139](https://github.com/myokit/myokit/pull/1139).

## Use in units

1. When comparing units with `==`, a check based on `eq` will be used: so units are considered equal if their exponents match to within machine precision.
2. The `Unit` class provides methods `close` and `close_exponent` which check if two units are within a certain tolerance (1e-9) of each other (`close`), or if they are within this tolerance of each other when omitting the multiplier (`close_exponent`, suggesting they can be converted into each other).
3. When taking a power of a unit, the exponents will be rounded to an integer _if and only if they are within machine precision of this integer_. This has no influence on numerics, but makes the result look like what we expect.

### TODO: Bit more on this
